In [ ]:
import numpy as np
import starry

np.random.seed(42)

starry.config.lazy = False

In [ ]:
%matplotlib inline
%run notebook_setup.py

In [ ]:
# Create a Starry map and place the two volcanos on the surface of Io
true_map = starry.Map(ydeg=30, reflected=True)

# Surt, Tvashtar Patera, Heno Patera, Loki patera, Pillan patera, Estan patera
spot_lats = np.array([41.0, 61.0, -56.0, 13.4, -12.0, 20.5])
spot_longs = 360.0 - np.array([340.0, 121.0, 307.2, 308.5, 243.5, 76.2])

for i in range(len(spot_lats)):
    true_map.add_spot(amp=0.01, sigma=0.01, lat=spot_lats[i], lon=spot_longs[i])

In [ ]:
360 - spot_longs

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
true_map.show(ax=ax, projection="rect", illuminate=False)

## Emitted light phase curves

In [ ]:
# Load design matrix from a previous notebook
A_phase_em = np.load("design_matrix_phase_em.npy")
np.shape(A_phase_em)

In [ ]:
# Compute model flux
flux = A_phase_em @ true_map.y

# Add some noise
flux_err = np.ones(len(flux)) * 1e-2 * np.std(flux)
flux_obs = flux + np.random.normal(0, flux_err, size=(len(flux)))

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(flux, "k.")
ax[1].plot(flux_obs, "k.")

In [ ]:
# Priors
ydeg = 8
map_phase_em = starry.Map(ydeg)
ncoeff = int((ydeg + 1) ** 2)

mu = np.empty(map_phase_em.Ny)
mu[0] = 1
mu[1:] = 0
L = np.empty(map_phase_em.Ny)
L[0] = 1e-1
L[1:] = 5e-4
map_phase_em.set_prior(L=L)

In [ ]:
%%time 
map_phase_em.set_prior(L=L)
map_phase_em.set_data(flux_obs, C=flux_err**2)
x, cho_cov = map_phase_em.solve(design_matrix=A_phase_em[:, :ncoeff])

In [ ]:
map_phase_em.amp = x[0]
map_phase_em[1:, :] = x[1:] / x[0]

In [ ]:
def plot_mean_map(ax, map, **kwargs):
    """
    Plots mean map in data space with transparency weighted by inverse
    variance.
    """
    minmax = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))

    # Draw samples from posterior
    images = []

    for i in range(100):
        map.draw()  # Draw a posterior sample
        img = map.render(projection="rect", **kwargs)
        images.append(img)

    mean_img = np.mean(images, axis=0)
    std_img = np.std(images, axis=0)

    inv_var = 1 / std_img ** 2
    alpha = minmax(inv_var)

    extent = (-180, 180, -90, 90)
    ax.imshow(mean_img, origin="lower", extent=extent, cmap="plasma", alpha=alpha)


fig, ax = plt.subplots(figsize=(10, 6))

plot_mean_map(ax, map_phase_em)

ax.set_title("Phase curves emitted light")

ax.set_ylabel("Lat")
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90])

ax.set_xlabel("Lon")
ax.set_xticks(np.arange(-180, 210, 30));

## Emitted light occultations

In [ ]:
# Load design matrix from a previous notebook
A_occ_em = np.load("design_matrix_occ_em.npy")
np.shape(A_occ_em)

In [ ]:
# Compute model flux
flux = A_occ_em @ true_map.y

# Add some noise
flux_err = np.ones(len(flux)) * 1e-2 * np.std(flux)
flux_obs = flux + np.random.normal(0, flux_err, size=(len(flux)))

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(flux, "k.")
ax[1].plot(flux_obs, "k.")

In [ ]:
# Priors
ydeg = 8
map_occ_em = starry.Map(ydeg)
ncoeff = int((ydeg + 1) ** 2)

mu = np.empty(map_occ_em.Ny)
mu[0] = 1
mu[1:] = 0
L = np.empty(map_occ_em.Ny)
L[0] = 1e-1
L[1:] = 5e-4
map_occ_em.set_prior(L=L)

In [ ]:
%%time 
map_occ_em.set_prior(L=L)
map_occ_em.set_data(flux_obs, C=flux_err**2)
x, cho_cov = map_occ_em.solve(design_matrix=A_occ_em[:, :ncoeff])

In [ ]:
map_occ_em.amp = x[0]
map_occ_em[1:, :] = x[1:] / x[0]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

plot_mean_map(ax, map_occ_em)

ax.set_title("Occultations in emitted light")

ax.set_ylabel("Lat")
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90])

ax.set_xlabel("Lon")
ax.set_xticks(np.arange(-180, 210, 30));

## Reflected light phase curves

In [ ]:
# Load design matrix from a previous notebook
A_phase_ref = np.load("design_matrix_phase_ref.npy")
np.shape(A_phase_ref)

In [ ]:
# Compute model flux
flux = A_phase_ref @ true_map.y

# Add some noise
flux_err = np.ones(len(flux)) * 1e-2 * np.std(flux)
flux_obs = flux + np.random.normal(0, flux_err, size=(len(flux)))

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(flux, "k.")
ax[1].plot(flux_obs, "k.")

In [ ]:
# Priors
ydeg = 8
map_phase_ref = starry.Map(ydeg, reflected=True)
ncoeff = int((ydeg + 1) ** 2)

mu = np.empty(map_phase_ref.Ny)
mu[0] = 1
mu[1:] = 0
L = np.empty(map_phase_ref.Ny)
L[0] = 1e-1
L[1:] = 5e-4
map_phase_ref.set_prior(L=L)

In [ ]:
%%time 
map_phase_ref.set_prior(L=L)
map_phase_ref.set_data(flux_obs, C=flux_err**2)
x, cho_cov = map_phase_ref.solve(design_matrix=A_phase_ref[:, :ncoeff])

In [ ]:
map_phase_ref.amp = x[0]
map_phase_ref[1:, :] = x[1:] / x[0]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

plot_mean_map(ax, map_phase_ref, illuminate=False)

ax.set_title("Phase curves in reflected light")

ax.set_ylabel("Lat")
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90])

ax.set_xlabel("Lon")
ax.set_xticks(np.arange(-180, 210, 30));

## Reflected light occultations

In [ ]:
# Load design matrix from a previous notebook
A_occ_ref = np.load("design_matrix_occ_ref.npy")
np.shape(A_occ_ref)

In [ ]:
# Compute model flux
flux = A_occ_ref @ true_map.y

# Add some noise
flux_err = np.ones(len(flux)) * 1e-2 * np.std(flux)
flux_obs = flux + np.random.normal(0, flux_err, size=(len(flux)))

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(flux, "k.")
ax[1].plot(flux_obs, "k.")

In [ ]:
# Priors
ydeg = 8
map_occ_ref = starry.Map(ydeg, reflected=True)
ncoeff = int((ydeg + 1) ** 2)

mu = np.empty(map_occ_ref.Ny)
mu[0] = 1
mu[1:] = 0
L = np.empty(map_occ_ref.Ny)
L[0] = 1e-1
L[1:] = 5e-4
map_occ_ref.set_prior(L=L)

In [ ]:
%%time 
map_occ_ref.set_prior(L=L)
map_occ_ref.set_data(flux_obs, C=flux_err**2)
x, cho_cov = map_occ_ref.solve(design_matrix=A_occ_ref[:, :ncoeff])

In [ ]:
map_occ_ref.amp = x[0]
map_occ_ref[1:, :] = x[1:] / x[0]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

plot_mean_map(ax, map_occ_ref, illuminate=False)

ax.set_title("Occultations in reflected light")

ax.set_ylabel("Lat")
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90])

ax.set_xlabel("Lon")
ax.set_xticks(np.arange(-180, 210, 30));

In [ ]:
# Plot true map
fig, ax = plt.subplots()
img = true_map.render(projection="rect", illuminate=False)
extent = (-180, 180, -90, 90)
ax.imshow(img, origin="lower", extent=extent, cmap="plasma")

ax.set_yticks([-90, -60, -30, 0, 30, 60, 90])
ax.set_xticks(np.arange(-180, 240, 60))
ax.set_title("True map")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 6))
fig.subplots_adjust(hspace=0.4, wspace=0.005)

plot_mean_map(ax[0, 0], map_phase_em)
plot_mean_map(ax[1, 0], map_occ_em)
plot_mean_map(ax[0, 1], map_phase_ref, illuminate=False)
plot_mean_map(ax[1, 1], map_occ_ref, illuminate=False)

ax[0, 0].set_title("Phase curves in emitted light")
ax[0, 1].set_title("Phase curves in reflected light")
ax[1, 0].set_title("Occultations in emitted light")
ax[1, 1].set_title("Occultations  in reflected light")

for a in ax[:, 1]:
    a.set_yticks([])
for a in ax[:, 0]:
    a.set_yticks([-90, -60, -30, 0, 30, 60, 90])
    a.set_ylabel("Latitude [deg]")
for a in ax[1, :]:
    a.set_xlabel("Longitude [deg]")


for i, s in enumerate(spot_longs):
    if s > 180:
        spot_longs[i] = -(360 - s)

for a in ax.flatten():
    a.set_xticks(np.arange(-180, 240, 60))
    for i in range(len(spot_lats)):
        a.scatter(spot_longs[i], spot_lats[i], color="black", marker="x", alpha=0.3)

## Mutual occultations


In [ ]:
# Load design matrix from a previous notebook
A_occ_ref_mut = np.load("design_matrix_mutual_ref.npy")
np.shape(A_occ_ref_mut)

In [ ]:
# Compute model flux
flux = A_occ_ref_mut @ true_map.y

# Add some noise
flux_err = np.ones(len(flux)) * 1e-2 * np.std(flux)
flux_obs = flux + np.random.normal(0, flux_err, size=(len(flux)))

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(flux, "k.")
ax[1].plot(flux_obs, "k.")

In [ ]:
# Priors
ydeg = 8
map_occ_ref_mut = starry.Map(ydeg, reflected=True)
ncoeff = int((ydeg + 1) ** 2)

mu = np.empty(map_occ_ref_mut.Ny)
mu[0] = 1
mu[1:] = 0
L = np.empty(map_occ_ref_mut.Ny)
L[0] = 1e-1
L[1:] = 5e-4
map_occ_ref.set_prior(L=L)

In [ ]:
%%time 
map_occ_ref_mut.set_prior(L=L)
map_occ_ref_mut.set_data(flux_obs, C=flux_err**2)
x, cho_cov = map_occ_ref_mut.solve(design_matrix=A_occ_ref_mut[:, :ncoeff])

In [ ]:
map_occ_ref_mut.amp = x[0]
map_occ_ref_mut[1:, :] = x[1:] / x[0]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

plot_mean_map(ax, map_occ_ref_mut, illuminate=False)

ax.set_title("Mutual occultations in reflected light")

ax.set_ylabel("Lat")
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90])

ax.set_xlabel("Lon")
ax.set_xticks(np.arange(-180, 210, 30))
for i in range(len(spot_lats)):
    ax.scatter(spot_longs[i], spot_lats[i], color="black", marker="x", alpha=0.3)